In [1]:
import requests
import re

def protocol_fenix_extraction():
    # La URL del objetivo que ya conocemos (o la ruta de tu USB)
    target_url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...")
    print(f"🎯 Objetivo: {target_url}")

    try:
        # 1. Descarga fresca para evitar NameErrors
        response = requests.get(target_url, timeout=15)
        if response.status_code != 200:
            print("❌ Error al acceder al archivo. La NSA podría estar interfiriendo.")
            return

        content = response.text
        print(f"✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...\n")

        # 2. Búsqueda de Hosts Internos (La llave del WAF)
        print(f"🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...")
        # Regex para dominios jpmorgan que parezcan internos o de API
        hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|stg|internal|auth)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", content, re.I)

        if hosts:
            unique_hosts = sorted(list(set(hosts)))
            print(f"🔥 HOSTS INTERNOS DETECTADOS ({len(unique_hosts)}):")
            for h in unique_hosts:
                print(f"   >>> 🌐 {h}")
        else:
            print("⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...")

        # 3. Búsqueda de Endpoints de Janus (El objetivo del ataque)
        print(f"\n🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...")
        endpoints = re.findall(r"[\"'](/api/[a-zA-Z0-9/_.-]*?(?:cookie|session|auth|janus)[a-zA-Z0-9/_.-]*)[\"']", content, re.I)

        if endpoints:
            unique_endpoints = sorted(list(set(endpoints)))
            print(f"🔥 RUTAS CRÍTICAS IDENTIFICADAS ({len(unique_endpoints)}):")
            for e in unique_endpoints[:10]: # Mostramos las top 10
                print(f"   >>> 🛣️ {e}")

    except Exception as e:
        print(f"⚠️ Error Crítico en el Protocolo Fénix: {e}")

# Ejecución Inmediata
protocol_fenix_extraction()

🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...
🎯 Objetivo: https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js
✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...

🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...
⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...

🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...
🔥 RUTAS CRÍTICAS IDENTIFICADAS (1):
   >>> 🛣️ /api/v0/cookie/validate/janus


In [2]:
import requests

def extract_janus_context():
    # Usamos la misma URL fuente ya que simulamos leer del USB
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos la posición exacta de la ruta crítica
        target = "/api/v0/cookie/validate/janus"
        index = content.find(target)

        if index != -1:
            # Extraemos 150 caracteres de contexto para ver la lógica
            start = max(0, index - 150)
            end = min(len(content), index + 150)
            snippet = content[start:end]

            print(f"🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:")
            print("-" * 60)
            # Resaltamos el objetivo para que el Triunfador lo vea claro
            print(f"...{snippet.replace(target, f' >> {target} << ')}...")
            print("-" * 60)

            # Análisis rápido de variables comunes
            if "concat" in snippet or "+" in snippet:
                print("💡 PISTA: Se está usando concatenación de cadenas.")
            if "http" in snippet:
                print("💡 PISTA: Hay un esquema hardcodeado cerca.")

        else:
            print("⚠️ La cadena exacta no se encontró (¿codificación diferente?).")

    except Exception as e:
        print(f"❌ Error en la biopsia: {e}")

extract_janus_context()

🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...
🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:
------------------------------------------------------------
...       btnGenerateClientSecret.attr('disabled', 'disabled');

                        try {
                            const response = await fetch(' >> /api/v0/cookie/validate/janus << ');

                            if (response.ok) {
                                window.open(`${DASHBOARD.GENERATE_CLI...
------------------------------------------------------------


In [3]:
import re
import requests

def extract_dashboard_object():
    # Usamos la fuente original (simulando el USB)
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos patrones de definición de objeto: "DASHBOARD={" o "DASHBOARD:{"
        # Capturamos hasta 500 caracteres después para ver las propiedades
        match = re.search(r"(?:const|var|let)\s+DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)

        if match:
            config_block = match.group(1)
            print(f"🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:")
            print("-" * 60)
            # Limpiamos un poco el formato para que el Triunfador lo lea bien
            print(config_block)
            print("-" * 60)

            if "GENERATE_CLI" in config_block:
                print("✅ CONFIRMADO: Contiene las rutas de generación de secretos.")
        else:
            # Intento secundario: a veces se define como propiedad de window
            match_window = re.search(r"window\.DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)
            if match_window:
                print(f"🔥 [!] OBJETO GLOBAL (WINDOW) INTERCEPTADO:")
                print(match_window.group(1))
            else:
                print("⚠️ No se encontró una definición explícita. Podría estar importado de otro módulo.")

    except Exception as e:
        print(f"❌ Error en la extracción: {e}")

extract_dashboard_object()

🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...
🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:
------------------------------------------------------------
{
    MANAGE_CLIENT_IDS_ENABLED: false,
    GENERATE_CLIENT_SECRET_URL: '',
    metadata: [],
    projects: [],
    selectedProject: {},

    //---> Create HTML elements
    createProjectRow: function (callback) {

        // Let's create a popup element
        var addProjectModal = $('<div>', {
            'class': 'dashboard-modal'
        }).append($('<label>', {
            'class': 'dashboard-modal-label',
            'text': 'Project name (required)'
        }
------------------------------------------------------------
✅ CONFIRMADO: Contiene las rutas de generación de secretos.
